> ### Note on Labs and Assignments:
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> These sections are graded and are not optional.
>

# IS 4487 Lab 6: Data Cleaning

## Outline

- Load and inspect a new dataset (Megatelco)
- Fix column names and data types
- Handle missing values
- Remove duplicate rows
- Review and remove outliers
- Reflect on data quality

In this lab, we’ll clean the data to get it ready for transformations and analysis.

We will continue working with this dataset in **Lab 7**, where we will create new features and apply transformations.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_06_data_cleaning.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do you use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/812e9f15c357a5657a2795631fcaa9d9363cb417/DataSets/megatelco_leave_survey_data_cleaning_v2.csv"
df = pd.read_csv(url)

df.head()

,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
0,one,403137.0,70,0.0,6605.0,199,841317,653.0,5.0,8.0,low,low,yes,LEAVE,8183,Android
1,zero,129700.0,67,16.0,6028.0,134,476664,1193.0,5.0,5.0,low,low,yes,LEAVE,12501,IOS
2,zero,69741.0,60,0.0,1482.0,176,810225,1037.0,3.0,8.0,low,low,yes,STAY,7425,IOS
3,one,377572.0,0,22.0,3005.0,184,826967,1161.0,0.0,5.0,low,low,no,LEAVE,13488,IOS
4,zero,382080.0,0,0.0,1794.0,74,951896,1023.0,0.0,14.0,low,low,yes,STAY,11389,IOS


In [2]:
# create a copy of your dataset for use in part 4
copied_df = df.copy(deep=True)

## Part 1: Review Column Names and Structure

Think about:

- Are column names consistent (lowercase, no spaces)?
- Are there any typos or redundant labels?
- Do the rows and columns appear aligned? (Are all the columns the same size? Are all the rows the same size?)

Why this matters:
Inconsistent or messy column names can break code and make analysis harder to follow.




In [3]:
# Standardize column names: lowercase, no spaces
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Get column info and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

In [4]:
# View descriptive statistics for numerical columns
df.describe()

,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,15006.000000,15016.000000,14916.000000,14916.000000,15016.000000,1.501600e+04,14916.000000,15013.00000,14916.000000,15016.000000
mean,242013.863455,153.430674,37.487664,4200.979686,135.946590,8.771293e+05,794.937249,10.56551,10.060941,11856.541289
std,109627.859666,113.019892,28.052318,2203.802446,62.934783,2.870168e+05,1238.997927,8.40421,41.188957,6812.183367
min,-65000.000000,0.000000,0.000000,400.000000,52.000000,-4.630000e+02,-200.000000,0.00000,1.000000,2.000000
25%,147818.500000,54.000000,12.000000,2292.750000,93.000000,6.444678e+05,498.000000,3.00000,5.000000,6135.000000
50%,241750.500000,151.000000,34.000000,4220.000000,135.000000,8.762530e+05,777.000000,9.00000,10.000000,11754.500000
75%,336442.000000,242.000000,62.000000,6079.250000,178.000000,1.098829e+06,1063.000000,17.00000,14.000000,17390.500000
max,432000.000000,380.000000,89.000000,8000.000000,5000.000000,1.456389e+06,125000.000000,35.00000,5000.000000,25354.000000


### Inspect categorical variables
Note that `df.describe()` only provides summary for numeric and date type variables. For variables defined as object - which are string/text, some maybe categorical (with limited and fixed number of allowed values), and others may be true string (can be any text, not limited in value).

For variables defined as object that we suspect are categorical you will often want to know what values are included. We can do this using `df[colname].value_counts()`

In [5]:
display(df['college'].value_counts())
display(df['reported_satisfaction'].value_counts())
display(df['reported_usage_level'].value_counts())
display(df['considering_change_of_plan'].value_counts())
display(df['operating_system'].value_counts())
display(df['leave'].value_counts())

,count
college,
zero,7960
one,7056


,count
reported_satisfaction,
low,10850
high,3415
avg,751


,count
reported_usage_level,
low,12235
high,2536
avg,245


,count
considering_change_of_plan,
yes,9267
no,4934


,count
operating_system,
Android,7813
IOS,7203


,count
leave,
STAY,7532
LEAVE,7484


## Part 2: Convert Data Types

Before analysis, make sure each column is stored in the correct format. This helps avoid calculation errors, makes plotting smoother, and ensures models interpret the data correctly.

Think about:
- Are numbers accidentally stored as strings?
- Should repeated text values be converted to categories?
- Are "yes"/"no" columns better represented as binary (0/1) or categorical types?

Fixing data types now saves time and avoids issues later in your workflow.




In [6]:
# Check original data types
print("Original dtypes:\n", df.dtypes)

# Convert categorical text columns
df['college'] = df['college'].astype('category')
df['reported_satisfaction'] = df['reported_satisfaction'].astype('category')
df['operating_system'] = df['operating_system'].astype('category')

# Convert object/text columns with limited possible values with an order to ordinal categorical columns
df['reported_satisfaction'] = pd.Categorical(df['reported_satisfaction'], categories = ['low', 'avg', 'high'], ordered = True)

# Convert binary columns ('yes'/'no', 'LEAVE'/'STAY') to binary categorical
df['considering_change_of_plan'] = df['considering_change_of_plan'].astype('category')

# Check updated data types
print("\nUpdated dtypes:\n", df.dtypes)


Original dtypes:
 college                         object
income                         float64
data_overage_mb                  int64
data_leftover_mb               float64
data_mb_used                   float64
text_message_count               int64
house                            int64
handset_price                  float64
over_15mins_calls_per_month    float64
average_call_duration          float64
reported_satisfaction           object
reported_usage_level            object
considering_change_of_plan      object
leave                           object
id                               int64
operating_system                object
dtype: object

Updated dtypes:
 college                        category
income                          float64
data_overage_mb                   int64
data_leftover_mb                float64
data_mb_used                    float64
text_message_count                int64
house                             int64
handset_price                   float64
over_1

### 🔧 Try It Yourself – Part 2

1. Convert the `leave` column from "yes"/"no" to binary (`1`/`0`) and make it a **category**
2. Convert `reported_usage_level` to an **ordinal categorical** type
3. Convert `house` to an **integer** type
3. Use `.info()` to confirm the changes


In [7]:
# add code here 🔧
df['leave'] = df['leave'].astype('category')
df['reported_usage_level'] = pd.Categorical(df['reported_usage_level'], categories = ['low', 'avg', 'high'], ordered = True)
df['house'] = df['house'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   college                      15016 non-null  category
 1   income                       15006 non-null  float64 
 2   data_overage_mb              15016 non-null  int64   
 3   data_leftover_mb             14916 non-null  float64 
 4   data_mb_used                 14916 non-null  float64 
 5   text_message_count           15016 non-null  int64   
 6   house                        15016 non-null  int64   
 7   handset_price                14916 non-null  float64 
 8   over_15mins_calls_per_month  15013 non-null  float64 
 9   average_call_duration        14916 non-null  float64 
 10  reported_satisfaction        15016 non-null  category
 11  reported_usage_level         15016 non-null  category
 12  considering_change_of_plan   14201 non-null  category
 13  l

## Part 3: Handle Missing Values

Missing data can break charts, skew stats, and disrupt models — so it needs to be handled carefully.

### Think about:
- Are the missing values random or patterned?
- Can we drop rows, or do we need to fill them?
- Should we use mean, median, or something else?

### Guidelines:
- Drop rows if there are only a few missing and the columns associated with them are essential to keep intact
- Use median to replace outliers in numeric columns
- Use 0 if the missing value means “none” (e.g. If the value was in response to: “do you have a history of chronic illness?” and the value was just left blank, we can assume that that blank just means “none” (the patient has no history of chronic illness))
- Use mode to replace categorical values

Cleaning missing values early avoids bigger problems later.

-----


**Note on `.loc` and Warnings** - When assigning values to a DataFrame, especially after filtering or copying, it's best to use `.loc` to avoid **`SettingWithCopyWarning`**. This ensures that you're updating the original data and not a temporary view of it.


In [8]:
# View missing value counts
print("Missing values per column:\n", df.isnull().sum())

# Fill 'handset_price' with median
df['handset_price'] = df['handset_price'].fillna(df['handset_price'].median())

# Drop rows with missing 'income' (if very few)
df = df.dropna(subset=['income']).copy()

# Fill missing 'data_leftover_mb' with 0 if it logically means no leftover data
df.loc[:, 'data_leftover_mb'] = df['data_leftover_mb'].fillna(0)

# Fill 'average_call_duration' with median if necessary
df.loc[:, 'average_call_duration'] = df['average_call_duration'].fillna(df['average_call_duration'].median())

# Fill 'data_mb_used' with median
df.loc[:, 'data_mb_used'] = df['data_mb_used'].fillna(df['data_mb_used'].median())

# Confirm updated missing values
print("\nMissing values after handling:\n", df.isnull().sum())


Missing values per column:
 college                          0
income                          10
data_overage_mb                  0
data_leftover_mb               100
data_mb_used                   100
text_message_count               0
house                            0
handset_price                  100
over_15mins_calls_per_month      3
average_call_duration          100
reported_satisfaction            0
reported_usage_level             0
considering_change_of_plan     815
leave                            0
id                               0
operating_system                 0
dtype: int64

Missing values after handling:
 college                          0
income                           0
data_overage_mb                  0
data_leftover_mb                 0
data_mb_used                     0
text_message_count               0
house                            0
handset_price                    0
over_15mins_calls_per_month      3
average_call_duration            0
reported_satisfa

### 🔧 Try It Yourself – Part 3


There are still some missing values in:

- `over_15mins_calls_per_month`
- `considering_change_of_plan`

Decide how to handle them based on what makes the most sense:

- Should you fill them with 0, the median, or something else?
- For categories, would a placeholder like "unknown" or the most common value work?
- Or is it better to drop those rows?

1. Write and execute code to handle the missing values in the remaining two columns.
2. Use `df.isnull().sum()` to confirm all missing values are handled.



In [14]:
# Add code here 🔧
df = df.dropna(subset=['over_15mins_calls_per_month']).copy()

# Correctly fill missing 'considering_change_of_plan' values with the mode
df.loc[:, 'considering_change_of_plan'] = df['considering_change_of_plan'].fillna(df['considering_change_of_plan'].mode()[0])

df.isnull().sum()

,0
college,0
income,0
data_overage_mb,0
data_leftover_mb,0
data_mb_used,0
text_message_count,0
house,0
handset_price,0
over_15mins_calls_per_month,0
average_call_duration,0


## Part 4: Remove Duplicate Rows

Sometimes the same row appears more than once due to data entry or processing mistakes. It's important to check for and remove these duplicates.

Think about:
- Are there rows that are exactly the same?
- If duplicates exist, should you keep the first one, the last one, or none?

Why this matters:
Duplicate rows can inflate totals, distort statistics, and lead to inaccurate conclusions.


In [15]:
# Check for exact duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Remove them, keeping the first occurrence
df = df.drop_duplicates()

# Confirm result
print(f"Remaining rows after removing duplicates: {len(df)}")

Number of duplicate rows: 17
Remaining rows after removing duplicates: 14986


### 🔧 Try It Yourself – Part 4

1. Use `copied_df.duplicated().sum()` to count how many duplicates are in your dataset.
2. Try using `copied_df.drop_duplicates(keep='last')` instead — what changes?

### In Your Response:
1. Explore whether duplicate rows share the same ID or just values across all columns and comment on your observation.


In [17]:
# 🔧 Add code here:
print(f"Number of duplicate rows: {copied_df.duplicated().sum()}")
copied_df = copied_df.drop_duplicates(keep='last')
print(f"Remaining rows after removing duplicates: {len(copied_df)}")

Number of duplicate rows: 17
Remaining rows after removing duplicates: 14999


### ✍️ Your Response: 🔧
1. There are more rows remaining. It appears that duplicate rows share the same values.

## Part 5: Identify and Remove Obvious Outliers

Outliers are values that fall far outside the normal range. They can come from data entry mistakes or rare cases.

- Use summary statistics or visual tools (like boxplots) to find them.
- Look for clearly unrealistic values — e.g., negative prices or extremely high data usage.
- Decide how to handle them:
  - Remove if they’re errors.
  - Keep if they’re valid but rare — or cap them if needed.

Outliers can distort averages, stretch visualizations, and mislead models, so it’s important to address them carefully.



In [18]:
# Remove negative or nonsensical values using business rules

# Example: remove rows where 'handset_price' is negative
df = df[df['handset_price'] >= 0]

# Example: remove rows with unusually long call durations
df = df[df['average_call_duration'] < 1000]

# Example: remove rows with extremely high text message counts
df = df[df['text_message_count'] < 1000]

# View shape after outlier filtering
print("Shape after removing obvious outliers:", df.shape)


Shape after removing obvious outliers: (14983, 16)


### 🔧 Try It Yourself – Part 5

1. Use `df.describe()` to look for columns with extreme minimum or maximum values.
2. Set a threshold for what you think is "too high" or "too low" for:
  - `data_mb_used`
  - `over_15mins_calls_per_month`
  - `income`
3. Remove those outliers using boolean filtering like `df = df[df['column'] < threshold]`

In [24]:
# 🔧 add code here:
df.describe()

df = df[df['data_mb_used'] < 4000]
df = df[df['over_15mins_calls_per_month'] < 10]
df = df[df['income'] < 100000]
df = df[df['income'] >= 0]

df.describe()

,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,428.000000,428.000000,428.000000,428.000000,428.000000,4.280000e+02,428.000000,428.000000,428.000000,428.000000
mean,76505.911215,116.682243,34.004673,2151.985981,134.077103,9.007763e+05,785.429907,3.341121,9.254673,11615.721963
std,13797.540293,113.027538,28.379578,1073.138814,49.517022,2.977140e+05,330.305369,2.662163,5.475353,6771.724254
min,52552.000000,0.000000,0.000000,403.000000,52.000000,-4.630000e+02,218.000000,0.000000,1.000000,6.000000
25%,64802.750000,21.000000,8.000000,1160.750000,92.000000,6.557662e+05,513.750000,1.000000,5.000000,6325.000000
50%,76819.000000,73.000000,28.000000,2116.000000,131.000000,9.179070e+05,777.000000,3.000000,9.000000,11634.000000
75%,89055.000000,199.250000,61.250000,3063.250000,178.000000,1.118875e+06,1082.250000,5.000000,14.000000,17025.000000
max,99609.000000,380.000000,89.000000,3995.000000,220.000000,1.453578e+06,1349.000000,9.000000,19.000000,25329.000000


## Part 6: Handle Outliers Using Quantiles

Instead of removing outliers, we can limit their impact by capping (removing) extreme values — a method known as **Winsorizing**.

### How to Do It:
- Use `.quantile()` to identify the 1st and 99th percentiles (or other thresholds).
- Use `.clip()` to cap values within that range.

This keeps your dataset intact while reducing the influence of extreme values on your analysis or model.



In [25]:
# Calculate 1st and 99th percentiles for income
income_min, income_max = df['income'].quantile([0.01, 0.99])

# Use .loc to avoid SettingWithCopyWarning and ensure assignment modifies the original DataFrame
df.loc[:, 'income'] = df['income'].clip(lower=income_min, upper=income_max)

# Clip 'data_mb_used' to within 1st and 99th percentiles
usage_min, usage_max = df['data_mb_used'].quantile([0.01, 0.99])
df.loc[:, 'data_mb_used'] = df['data_mb_used'].clip(lower=usage_min, upper=usage_max)

# Clip 'average_call_duration' to reduce the effect of extreme outliers
call_min, call_max = df['average_call_duration'].quantile([0.01, 0.99])
df.loc[:, 'average_call_duration'] = df['average_call_duration'].clip(lower=call_min, upper=call_max)



### 🔧 Try It Yourself – Part 6

1. Use `.quantile([0.01, 0.99])` to find the range for:
  - `text_message_count`
  - `over_15mins_calls_per_month`
2. Apply `.clip(lower=..., upper=...)` to reduce the impact of those outliers

### In Your Response:
1. Compare the `.describe()` output before and after clipping and comment on what you observe


In [32]:
from IPython.utils import text
# 🔧 Add code here
text_min, text_max = df['text_message_count'].quantile([0.01, 0.99])
df.loc[:, 'text_message_count'] = df['text_message_count'].clip(lower=text_min, upper=text_max)
over_15mins_calls_per_month_min, over_15mins_calls_per_month_max = df['over_15mins_calls_per_month'].quantile([0.01, 0.99])
df.loc[:, 'over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].clip(lower=over_15mins_calls_per_month_min, upper=over_15mins_calls_per_month_max)
df.describe()

,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,428.000000,428.000000,428.000000,428.000000,428.000000,4.280000e+02,428.000000,428.000000,428.000000,428.000000
mean,76506.884229,116.682243,34.004673,2151.852921,134.072430,9.007763e+05,785.429907,3.341121,9.254673,11615.721963
std,13785.935366,113.027538,28.379578,1072.519173,49.493139,2.977140e+05,330.305369,2.662163,5.475353,6771.724254
min,53240.290000,0.000000,0.000000,418.350000,53.000000,-4.630000e+02,218.000000,0.000000,1.000000,6.000000
25%,64802.750000,21.000000,8.000000,1160.750000,92.000000,6.557662e+05,513.750000,1.000000,5.000000,6325.000000
50%,76819.000000,73.000000,28.000000,2116.000000,131.000000,9.179070e+05,777.000000,3.000000,9.000000,11634.000000
75%,89055.000000,199.250000,61.250000,3063.250000,178.000000,1.118875e+06,1082.250000,5.000000,14.000000,17025.000000
max,99207.600000,380.000000,89.000000,3951.460000,219.000000,1.453578e+06,1349.000000,9.000000,19.000000,25329.000000


### ✍️ Your Response: 🔧
1. There honestly doesn't seem to be that much of a difference. Maybe that the range is smaller. There aren't any horrible outliers so I think the data is already quite nice the way it is.

## 🔧 Part 7: Reflection (100 words or less per question)

1. Which step fixed the most issues in the dataset?
2. What surprised you about the structure or values?
3. Do you feel this data is now ready for transformation in Lab 7?


### ✍️ Your Response: 🔧
1. I think that removing or filling in null/duplicate values is such a game changer for the dataset.
2. It surprised me that there are certain ways to handle categorical values compared to more numerical ones. I'm also a bit surprised about how much these extra values really impact the dataset.
3.  This lab is now ready for transformation in Lab 7 because it has been cleaned throughly and ready to be examined and utilized to make conclusions and models. We are ready to really take the data and make conclusions.

## Export Your Notebook to Submit in Canvas
- Use the instructions from Lab 1

In [33]:
!jupyter nbconvert --to html "lab_06_GiangBreanna.ipynb"

[NbConvertApp] Converting notebook lab_06_GiangBreanna.ipynb to html
[NbConvertApp] Writing 367098 bytes to lab_06_GiangBreanna.html
